# QuadPay: feature_engineering

#### Overview
__Goal.__ Apply feature engineering on data from `orders.csv`. Models will be trained at the customer-order level. 

__Approach__<br>
We've calculated cumulative metrics by customer_id and merchant_id, since we would like to compare aggregate metrics at these levels relative to the customer-order level. We use cumulative sum since we want to include transactions that occur up to a given point in time. We will aggregate data across these entities:
+ customer-order
+ customer
+ merchants
+ population

<br>

__Transformations Applied__
+ get customer cumulative data
+ get merchant cumulative data
+ get numeric transformations (eg. logs)
+ get date transformations

<br>

__Other Transformations to Consider__
+ one hot encoding
+ add population agg metrics
+ bucket by customer age
+ add zipcode median income, avg employment

<br>

#### Resources
+ [QuadPay Machine Learning Engineer Assignment](https://docs.google.com/document/d/1DUixAKGXQqFwhAIL7Sk-yhE_TqM7TovFnMyztVbMGNM/edit)
+ [Zip Code Data](https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2016-zip-code-data-soi)

<br>


In [1]:
# !pip install -r ../requirements.txt
import sys
sys.path.append('/home/jovyan/quadpay_analysis/src')


In [2]:
import pandas as pd
import numpy as np
import functions as fun

ORDERS_DATA = '../data/orders.csv'


### Get orders data

In [3]:
orders = pd.read_csv(ORDERS_DATA)
orders['orders'] = 1
orders.head()


,order_id,customer_id,merchant_id,order_amount,checkout_started_at,credit_decision_started_at,approved_for_installments,customer_credit_score,customer_age,customer_billing_zip,customer_shipping_zip,paid_installment_1,paid_installment_2,paid_installment_3,paid_installment_4,orders
0,42C39B,9D74A0,6881F9,147.20,2018-10-03 14:26:18.497,2018-10-03 14:26:18.997,True,626,26,95407,95407,1.0,1,1.0,1.0,1
1,A19AC4,2629AB,C37828,69.95,2018-09-21 16:55:03.443,2018-09-21 16:55:05.100,True,603,20,85007,85007,1.0,1,1.0,1.0,1
2,381AFF,793FE0,2526E8,45.00,2018-09-15 13:41:46.103,2018-09-15 13:41:47.197,True,648,41,30305-2646,30305-2646,1.0,1,1.0,1.0,1
3,3184A3,22C696,24FBAF,164.00,2018-10-10 00:59:25.793,2018-10-10 00:59:26.123,True,625,33,28562,28562,1.0,1,1.0,1.0,1
4,237711,D4E94A,6F498E,69.17,2018-09-12 22:53:08.970,2018-09-12 22:53:10.923,True,438,18,92324,92324,1.0,1,1.0,1.0,1


<br> 

### Get customer data
Get cumulative metrics by `customer_id`. We will train models at the customer-order level.

In [4]:
# get cumulative sum metrics
orders = fun.get_cumsum_by(
    df = orders, 
    by = ['customer_id','checkout_started_at'], 
    values = [
        'order_amount', 
        'orders', 
    ])

# get cumulative avg metrics
orders = fun.get_cumavg_by(
    df = orders, 
    by = ['customer_id','checkout_started_at'], 
    values = [
        'order_amount', 
        'paid_installment_1',
        'paid_installment_2',
        'paid_installment_3',
        'paid_installment_4',
    ])

orders.head()


,order_id,customer_id,merchant_id,order_amount,checkout_started_at,credit_decision_started_at,approved_for_installments,customer_credit_score,customer_age,customer_billing_zip,...,paid_installment_3,paid_installment_4,orders,order_amount_cum_sum,orders_cum_sum,order_amount_cum_avg,paid_installment_1_cum_avg,paid_installment_2_cum_avg,paid_installment_3_cum_avg,paid_installment_4_cum_avg
0,E4BBCD,0005F1,8B9AB8,169.00,2018-08-30 22:39:26.397,2018-08-30 22:39:27.757,True,711,25,34266,...,1.0,1.0,1,169.00,1,169.00,1.0,1,1.0,1.0
1,E128F5,0005F1,8B9AB8,297.00,2018-09-27 13:58:34.243,2018-09-27 13:58:34.523,True,711,25,34266,...,1.0,1.0,1,297.00,1,297.00,1.0,1,1.0,1.0
2,A39F4B,0008C3,8D665E,109.00,2018-09-08 04:20:36.700,2018-09-08 04:20:37.950,True,588,35,89183,...,1.0,1.0,1,109.00,1,109.00,1.0,1,1.0,1.0
3,BFFC42,000A1F,944AC9,1036.78,2018-09-28 15:38:18.650,2018-09-28 15:38:18.947,True,624,21,16117,...,1.0,1.0,1,1036.78,1,1036.78,1.0,1,1.0,1.0
4,A91B2B,000B25,26ECA2,58.06,2018-09-11 02:20:55.423,2018-09-11 02:20:56.530,True,611,19,32817,...,1.0,1.0,1,58.06,1,58.06,1.0,1,1.0,1.0


<br> 

### Get population data (WIP)
Get cumulative metrics for all customers.


In [15]:
# get cumulative avg metrics
population = orders[[ 
    'checkout_started_at', 
    'order_amount', 
    'order_amount_cum_sum',
    'order_amount_cum_avg', 
]]

population.head()


,checkout_started_at,order_amount,order_amount_cum_sum,order_amount_cum_avg
0,2018-08-30 22:39:26.397,169.00,169.00,169.00
1,2018-09-27 13:58:34.243,297.00,297.00,297.00
2,2018-09-08 04:20:36.700,109.00,109.00,109.00
3,2018-09-28 15:38:18.650,1036.78,1036.78,1036.78
4,2018-09-11 02:20:55.423,58.06,58.06,58.06


<br>

### Get merchant metrics
Get cumulative metrics by `merchant_id`.

In [5]:

# limit to desired columns 
merchants = orders[[
    'merchant_id',
    'checkout_started_at',
    'orders',
    'order_amount', 
    'paid_installment_1',
    'paid_installment_2',
    'paid_installment_3',
    'paid_installment_4',
]]

# get cumulative sums
merchants = fun.get_cumsum_by(
    df = merchants, 
    by = ['merchant_id','checkout_started_at'], 
    values = ['order_amount', 'orders', ])

# get cumulative avgs
merchants = fun.get_cumavg_by(
    df = merchants, 
    by = ['merchant_id','checkout_started_at'], 
    values = [
        'order_amount', 
        'paid_installment_1',
        'paid_installment_2',
        'paid_installment_3',
        'paid_installment_4',
    ])

# specify columns to keep
merchants_columns = {
    'merchant_id' : 'merchant_id',
    'checkout_started_at' : 'checkout_started_at',
    'order_amount_cum_sum' : 'merchant_order_amount_cum_sum',
    'orders_cum_sum' : 'merchant_orders_cum_sum',
    'order_amount_cum_avg' : 'merchant_order_amount_cum_avg',
    'paid_installment_1_cum_avg' : 'merchant_paid_installment_1_ratio',
    'paid_installment_2_cum_avg' : 'merchant_paid_installment_2_ratio',
    'paid_installment_3_cum_avg' : 'merchant_paid_installment_3_ratio',
    'paid_installment_4_cum_avg' : 'merchant_paid_installment_4_ratio',
}

# keep those columns and rename accordingly
merchants = merchants[merchants_columns.keys()]
merchants = merchants.rename(columns=merchants_columns)
merchants.head()


,merchant_id,checkout_started_at,merchant_order_amount_cum_sum,merchant_orders_cum_sum,merchant_order_amount_cum_avg,merchant_paid_installment_1_ratio,merchant_paid_installment_2_ratio,merchant_paid_installment_3_ratio,merchant_paid_installment_4_ratio
0,01B43C,2018-09-10 00:38:21.067,310.0,1,310.0,1.0,1,1.0,1.0
1,01B43C,2018-09-11 20:00:33.477,175.0,1,175.0,1.0,1,1.0,1.0
2,01B43C,2018-09-14 01:49:59.393,225.0,1,225.0,1.0,1,1.0,1.0
3,01B43C,2018-10-13 16:14:28.193,275.0,1,275.0,1.0,1,1.0,0.0
4,01C9EC,2018-08-28 23:34:39.247,104.0,1,104.0,1.0,1,1.0,1.0


In [6]:
# merge merchant level metrics to orders
orders = orders.merge(
    merchants,
    how='left',
    on=['merchant_id','checkout_started_at']
)

orders.head()


,order_id,customer_id,merchant_id,order_amount,checkout_started_at,credit_decision_started_at,approved_for_installments,customer_credit_score,customer_age,customer_billing_zip,...,paid_installment_2_cum_avg,paid_installment_3_cum_avg,paid_installment_4_cum_avg,merchant_order_amount_cum_sum,merchant_orders_cum_sum,merchant_order_amount_cum_avg,merchant_paid_installment_1_ratio,merchant_paid_installment_2_ratio,merchant_paid_installment_3_ratio,merchant_paid_installment_4_ratio
0,E4BBCD,0005F1,8B9AB8,169.00,2018-08-30 22:39:26.397,2018-08-30 22:39:27.757,True,711,25,34266,...,1,1.0,1.0,169.00,1,169.00,1.0,1,1.0,1.0
1,E128F5,0005F1,8B9AB8,297.00,2018-09-27 13:58:34.243,2018-09-27 13:58:34.523,True,711,25,34266,...,1,1.0,1.0,297.00,1,297.00,1.0,1,1.0,1.0
2,A39F4B,0008C3,8D665E,109.00,2018-09-08 04:20:36.700,2018-09-08 04:20:37.950,True,588,35,89183,...,1,1.0,1.0,109.00,1,109.00,1.0,1,1.0,1.0
3,BFFC42,000A1F,944AC9,1036.78,2018-09-28 15:38:18.650,2018-09-28 15:38:18.947,True,624,21,16117,...,1,1.0,1.0,1036.78,1,1036.78,1.0,1,1.0,1.0
4,A91B2B,000B25,26ECA2,58.06,2018-09-11 02:20:55.423,2018-09-11 02:20:56.530,True,611,19,32817,...,1,1.0,1.0,58.06,1,58.06,1.0,1,1.0,1.0


<br>

### Add numeric transformations
Add numeric transformations such as logs and ratios. 


In [7]:

# orders['order_amount_to_customer_avg_ratio'] = orders.order_amount / orders.order_amount_cum_avg
# orders['order_amount_to_merchant_avg_ratio'] = orders.order_amount / orders.merchant_order_amount_cum_avg


orders['log_order_amount'] = [ np.log(row) for row in orders['order_amount'] ]
orders['log_order_amount_cum_sum'] = [ np.log(row) for row in orders['order_amount_cum_sum'] ]
orders['log_order_amount_cum_avg'] = [ np.log(row) for row in orders['order_amount_cum_avg'] ]

orders.head()



,order_id,customer_id,merchant_id,order_amount,checkout_started_at,credit_decision_started_at,approved_for_installments,customer_credit_score,customer_age,customer_billing_zip,...,merchant_order_amount_cum_sum,merchant_orders_cum_sum,merchant_order_amount_cum_avg,merchant_paid_installment_1_ratio,merchant_paid_installment_2_ratio,merchant_paid_installment_3_ratio,merchant_paid_installment_4_ratio,log_order_amount,log_order_amount_cum_sum,log_order_amount_cum_avg
0,E4BBCD,0005F1,8B9AB8,169.00,2018-08-30 22:39:26.397,2018-08-30 22:39:27.757,True,711,25,34266,...,169.00,1,169.00,1.0,1,1.0,1.0,5.129899,5.129899,5.129899
1,E128F5,0005F1,8B9AB8,297.00,2018-09-27 13:58:34.243,2018-09-27 13:58:34.523,True,711,25,34266,...,297.00,1,297.00,1.0,1,1.0,1.0,5.693732,5.693732,5.693732
2,A39F4B,0008C3,8D665E,109.00,2018-09-08 04:20:36.700,2018-09-08 04:20:37.950,True,588,35,89183,...,109.00,1,109.00,1.0,1,1.0,1.0,4.691348,4.691348,4.691348
3,BFFC42,000A1F,944AC9,1036.78,2018-09-28 15:38:18.650,2018-09-28 15:38:18.947,True,624,21,16117,...,1036.78,1,1036.78,1.0,1,1.0,1.0,6.943875,6.943875,6.943875
4,A91B2B,000B25,26ECA2,58.06,2018-09-11 02:20:55.423,2018-09-11 02:20:56.530,True,611,19,32817,...,58.06,1,58.06,1.0,1,1.0,1.0,4.061477,4.061477,4.061477


<br>

### Add date transformations


In [11]:
orders['checkout_started_datetime'] = [ fun.date_string_to_time(row) for row in orders['checkout_started_at'] ]
orders['checkout_month_num'] = [ row.month for row in orders['checkout_started_datetime']]
orders['checkout_hour_num'] = [ row.hour for row in orders['checkout_started_datetime']]
orders['checkout_weekday_num'] = [ row.weekday() for row in orders['checkout_started_datetime']]
orders['checkout_is_weekend'] = [ row.weekday() in [5,6] for row in orders['checkout_started_datetime']]

orders.head(10)


,order_id,customer_id,merchant_id,order_amount,checkout_started_at,credit_decision_started_at,approved_for_installments,customer_credit_score,customer_age,customer_billing_zip,...,merchant_paid_installment_3_ratio,merchant_paid_installment_4_ratio,log_order_amount,log_order_amount_cum_sum,log_order_amount_cum_avg,checkout_started_datetime,checkout_month_num,checkout_weekday_num,checkout_is_weekend,checkout_hour_num
0,E4BBCD,0005F1,8B9AB8,169.00,2018-08-30 22:39:26.397,2018-08-30 22:39:27.757,True,711,25,34266,...,1.0,1.0,5.129899,5.129899,5.129899,2018-08-30 22:39:26,8,3,False,22
1,E128F5,0005F1,8B9AB8,297.00,2018-09-27 13:58:34.243,2018-09-27 13:58:34.523,True,711,25,34266,...,1.0,1.0,5.693732,5.693732,5.693732,2018-09-27 13:58:34,9,3,False,13
2,A39F4B,0008C3,8D665E,109.00,2018-09-08 04:20:36.700,2018-09-08 04:20:37.950,True,588,35,89183,...,1.0,1.0,4.691348,4.691348,4.691348,2018-09-08 04:20:36,9,5,True,4
3,BFFC42,000A1F,944AC9,1036.78,2018-09-28 15:38:18.650,2018-09-28 15:38:18.947,True,624,21,16117,...,1.0,1.0,6.943875,6.943875,6.943875,2018-09-28 15:38:18,9,4,False,15
4,A91B2B,000B25,26ECA2,58.06,2018-09-11 02:20:55.423,2018-09-11 02:20:56.530,True,611,19,32817,...,1.0,1.0,4.061477,4.061477,4.061477,2018-09-11 02:20:55,9,1,False,2
5,E68844,000BD5,E3FFE5,195.14,2018-09-21 13:40:35.743,2018-09-21 13:40:37.227,True,618,36,31324,...,1.0,1.0,5.273717,5.273717,5.273717,2018-09-21 13:40:35,9,4,False,13
6,B8EB7A,000D24,8B9AB8,207.00,2018-09-16 15:05:18.953,2018-09-16 15:05:20.250,True,543,43,84094,...,1.0,1.0,5.332719,5.332719,5.332719,2018-09-16 15:05:18,9,6,True,15
7,CE3D89,000EEC,0ADC71,108.49,2018-09-29 15:52:47.543,2018-09-29 15:52:47.967,True,550,44,29223,...,1.0,1.0,4.686658,4.686658,4.686658,2018-09-29 15:52:47,9,5,True,15
8,F71935,001368,C37828,49.85,2018-09-07 22:30:22.447,2018-09-07 22:30:24.150,True,555,29,33472,...,1.0,1.0,3.909018,3.909018,3.909018,2018-09-07 22:30:22,9,4,False,22
9,76AC92,001FDA,0B3A54,227.00,2018-10-14 17:32:21.170,2018-10-14 17:32:21.523,True,588,23,01841,...,1.0,1.0,5.424950,5.424950,5.424950,2018-10-14 17:32:21,10,6,True,17


<br>

### Write new features to CSV
We'll do this to separate the ETL/feature engineering from the data viz.

In [9]:
orders.to_csv('../data/transformed_orders.csv', index=False)


In [10]:
list(orders)


['order_id',
 'customer_id',
 'merchant_id',
 'order_amount',
 'checkout_started_at',
 'credit_decision_started_at',
 'approved_for_installments',
 'customer_credit_score',
 'customer_age',
 'customer_billing_zip',
 'customer_shipping_zip',
 'paid_installment_1',
 'paid_installment_2',
 'paid_installment_3',
 'paid_installment_4',
 'orders',
 'order_amount_cum_sum',
 'orders_cum_sum',
 'order_amount_cum_avg',
 'paid_installment_1_cum_avg',
 'paid_installment_2_cum_avg',
 'paid_installment_3_cum_avg',
 'paid_installment_4_cum_avg',
 'merchant_order_amount_cum_sum',
 'merchant_orders_cum_sum',
 'merchant_order_amount_cum_avg',
 'merchant_paid_installment_1_ratio',
 'merchant_paid_installment_2_ratio',
 'merchant_paid_installment_3_ratio',
 'merchant_paid_installment_4_ratio',
 'log_order_amount',
 'log_order_amount_cum_sum',
 'log_order_amount_cum_avg',
 'checkout_started_datetime',
 'checkout_month_num',
 'checkout_weekday_num',
 'checkout_is_weekend']